# Distance matrices

In this notebook, we will  create the distance matrices. We will have to matrices: one for gas cars ane electric cars and the other one for drones. The distance matrices will be used to calculate the cost of the routes.

Each index will be a coordinate in the grid. The distance between two coordinates will be the Havesian distance between them (in the case of drones) and the one given by OSRM (in the case of vehicles).

In [4]:
import pandas as pd
import numpy as np
import requests

# Data for Gas Cars
gas_cars = {
    "Vehicle": "Gas Car",
    "Freight Rate [COP/km]": 5000,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 30000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": 16000,
    "Recharge/Fuel Time [min/10 percent charge]": 0.1,
    "Avg. Speed [km/h]": None,
    "Gas Efficiency [km/gal]": 10,
}

# Data for Drones
drones = {
    "Vehicle": "Drone",
    "Freight Rate [COP/km]": 500,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 3000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": 220.73,
    "Recharge/Fuel Time [min/10 percent charge]": 2,
    "Avg. Speed [km/h]": 40,
    "Gas Efficiency [km/gal]": None,
    "Electricity Efficency [kWh/km]": 0.15,
}


# Data for Solar EVs
solar_ev = {
    "Vehicle": "Solar EV",
    "Freight Rate [COP/km]": 4000,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 21000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": None,
    "Recharge/Fuel Time [min/10 percent charge]": None,
    "Avg. Speed [km/h]": None,
    "Gas Efficiency [km/gal]": None,
    "Electricity Efficency [kWh/km]": 0.15,
}



In [14]:
import math


depots_df = pd.read_csv('Depots.csv', delimiter=";|,", engine='python')
clients_df = pd.read_csv('Clients.csv', delimiter= ";|,", engine='python')


# extract coordinates
depots_coords = depots_df[['Longitude', 'Latitude']].values.tolist()
clients_coords = clients_df[['Longitude', 'Latitude']].values.tolist()

# Combine depots and clients
all_coords = depots_coords + clients_coords

# Build the coordinates string for OSRM
coords_str = ';'.join([f"{lon},{lat}" for lon, lat in all_coords])




# API URL
url = f"https://router.project-osrm.org/table/v1/driving/{coords_str}"

# Parameters
params = {
    'sources': ';'.join(map(str, range(len(all_coords)))),
    'destinations': ';'.join(map(str, range(len(all_coords)))),
    'annotations': 'duration,distance'
}

# Send the request
response = requests.get(url, params=params)

# Check for successful response
if response.status_code != 200:
    print(f"Error: {response.status_code}")
    print(response.text)
    exit()

data = response.json()

# Extract the distance matrix
car_matrix_distance = np.array(data['distances'])

# Extract the duration matrix
car_matrix_time = np.array(data['durations'])



def haversine(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

nodes = all_coords

print(nodes)

drone_matrix = [[0 for _ in range(len(nodes))] for _ in range(len(nodes))]


for i in range(len(nodes)):
    for j in range(len(nodes)):
        if i != j:
            origin = nodes[i]
            destination = nodes[j]
            origin_latitud = origin[1]
            origin_longitud = origin[0]
            destination_latitud = destination[1]
            destination_longitud = destination[0]
            drone_matrix[i][j] = haversine(origin_longitud, origin_latitud, destination_longitud, destination_latitud)



cars_df = pd.read_csv("multi_vehicles.csv", delimiter = ";|,", engine='python')
cars = cars_df.to_dict()

clients_df = pd.read_csv("Clients.csv", delimiter = ";|,", engine='python')
clients = clients_df.to_dict()

num_depots = len(depots_coords)

num_clients = len(clients_coords)

num_cars = len(cars["Range"])

[[-74.08124218159384, 4.75021190869025], [-74.10993358606953, 4.5363832206427785], [-74.03854814565923, 4.792925960208614], [-74.06706883098641, 4.72167778077445], [-74.13826337931849, 4.607707046760958], [-74.12400186370824, 4.650463053612691], [-74.09561875464892, 4.621911772492814], [-74.10975623736951, 4.678960680833056], [-74.09547235719887, 4.735973062153282], [-74.10991610076434, 4.550640992537941], [-74.10977422186991, 4.664702960902753], [-74.12408925943333, 4.5791740634103135], [-74.12918518, 4.596339809], [-74.02342238, 4.755848993], [-74.06261629, 4.737073418], [-74.09198006, 4.718090836], [-74.09907444, 4.599326325], [-74.12984965, 4.574080572], [-74.14887861, 4.623949992], [-74.08950946, 4.592782485], [-74.13207707, 4.564187203], [-74.07650203, 4.697451423], [-74.13930564, 4.636023168], [-74.11858872, 4.618693336], [-74.1323996, 4.654261512], [-74.03391849, 4.699785416], [-74.07786788, 4.680270425], [-74.11684813, 4.705858261], [-74.07257443, 4.750004017], [-74.09599058, 

In [ ]:
from __future__ import division
from pyomo.environ import *
import pandas  as pd
from pyomo.opt import SolverFactory
import networkx as nx
from matplotlib import pyplot as plt

class BaseCase:
    def __init__(self, gas_cars, drones, solar_ev, car_matrix_distance, car_matrix_time, drone_matrix, clients, num_depots, num_clients, num_cars, cars):
        """
        Initialize the model parameters.
        """
        self.gas_cars = gas_cars
        self.drones = drones
        self.solar_ev = solar_ev
        self.car_matrix_distance = car_matrix_distance
        self.car_matrix_time = car_matrix_time
        self.drone_matrix = drone_matrix
        self.clients = clients
        self.num_clients = num_clients
        self.num_depots = num_depots
        self.num_cars = num_cars
        self.cars = cars #This will be a dictionary with the data of the cars. The keys will be the car number and the values will be the data of the car.


        # Create the Pyomo model
        self.model = ConcreteModel()

    def build_model(self):
        """
        Build the optimization model.
        """
        model = self.model

        # Sets
        model.depots_set = RangeSet(0, self.num_depots-1) #Set of depots. In the cost_matrix, the depots come first  
        model.clients_set = RangeSet(self.num_depots, self.num_clients+self.num_depots-1) #Set of clients. In the cost matrix, the clients come after the depots.
        model.nodes = RangeSet(0, self.car_matrix_distance.size -1) #Set of nodes. This includes both depots and clients.
        model.cars_set = RangeSet(0, self.num_cars-1) #Set of vehicles.
        
        #Helper Sets
        model.U = RangeSet(0, len(model.clients_set)-1) #For the weird condition


        # Decision Variables
        model.x = Var(model.nodes, model.nodes, model.cars_set, domain=Binary) #Decision variables specifying if a given car goes from node i to node j
        model.u = Var(model.clients_set, model.cars_set, domain=NonNegativeReals, bounds = (0, len(model.nodes)-1))


        # Objective: Minimize total cost (energy, installation, and communication costs)
        def obj_expression(model):
            #Cost of loading a car
            cost1 = sum((sum(self.clients["Product"][i-self.num_depots]*model.x[i,j,k] for i in model.clients_set)/2)*500 for k in model.cars_set)

            #Cost of driving the car (distance)
            cost2 = 0
            for i in model.nodes:
                for j in model.nodes:
                    for k in model.cars_set:
                        if self.cars["VehicleType"][k] == "Gas Car":
                            cost2 += self.car_matrix_distance[i][j]*model.x[i,j,k]*self.gas_cars["Freight Rate [COP/km]"]
                        elif self.cars["VehicleType"][k] == "Drone":
                            cost2 += self.drone_matrix[i][j]*model.x[i,j,k]*self.drones["Freight Rate [COP/km]"]
                        elif self.cars["VehicleType"][k] == "Solar EV":
                            cost2 += self.car_matrix_distance[i][j]*model.x[i,j,k]*self.solar_ev["Freight Rate [COP/km]"]

            #Cost of driving the car (time)
            cost3 = 0
            for i in model.nodes:
                for j in model.nodes:
                    for k in model.cars_set:
                        if self.cars["VehicleType"][k] == "Gas Car":
                            cost3 += self.car_matrix_time[i][j]*model.x[i,j,k]*self.gas_cars["Time Rate [COP/min]"]
                        elif self.cars["VehicleType"][k] == "Drone":
                            cost3 += self.drone_matrix[i][j]*model.x[i,j,k]*self.drones["Time Rate [COP/min]"]
                        elif self.cars["VehicleType"][k] == "Solar EV":
                            cost3 += self.car_matrix_time[i][j]*model.x[i,j,k]*self.solar_ev["Time Rate [COP/min]"]
                            
            #Cost for gas
            cost4 = 0
            for k in model.cars_set:
                if self.cars["VehicleType"][k] == "Gas Car":
                    cost4 += self.gas_cars["Recharge/Fuel Cost [COP/(gal or kWh)]"]*sum(self.car_matrix_time[i][j]*model.x[i,j,k] for i in model.nodes for j in model.nodes) / self.gas_cars["Gas Efficiency [km/gal]"]
                elif self.cars["VehicleType"][k] == "Drone":
                    cost4 += self.drones["Recharge/Fuel Cost [COP/(gal or kWh)]"]*sum(self.car_matrix_time[i][j]*model.x[i,j,k] for i in model.nodes for j in model.nodes) / self.drones["Electricity Efficency [kWh/km]"]
                elif self.cars["VehicleType"][k] == "Solar EV":
                    cost4 += 0
            #Cost for maintenance
            cost5 = 0
            for k in model.cars_set:
                cost5 += self.cars["Daily Maintenance"][k]

            return cost1 + cost2 + cost3 + cost4 + cost5

        model.obj = Objective(rule=obj_expression, sense=minimize)

        # Constraint1.1: Each client is visited exactly once. Each client is entered by a car.
        def restriccion_1_1(model, i):
            return sum(model.x[i,j,k] for j in model.clients_set for k in model.cars_set if i != j) == 1
        model.restriccion_1_1 = Constraint(model.clients_set,  rule = restriccion_1_1)

        # Constraint1.2: Each client is exited exactly once. Each client is exited by a car.
        def restriccion_1_2(model, j):
            return sum(model.x[i,j,k] for i in model.clients_set for k in model.cars_set if i != j) == 1
        model.restriccion_1_2 = Constraint(model.clients_set,  rule = restriccion_1_2)
        
        # Constraint2.1: Each car should exit from a depot.
        def restriccion_2_1(model, k, i):
            return sum(model.x[i,j,k] for j in model.depots_set) == len(self.cars['VehicleType'])
        model.restriccion_2_1 = Constraint(model.cars_set, model.depots_set, rule = restriccion_2_1)

        #Constraint2.2: Each car should enter a depot.
        def restriccion2_2(model, k, j):
            return sum(model.x[i,j,k] for i in model.depots_set) == len(self.cars['VehicleType'])
        model.restriccion_2_2 = Constraint(model.cars_set, model.depots_set, rule = restriccion2_2)

        #Constraint3: Each client should be visited exactly once.
        def restriccion_3(model, i, k):
            return sum(model.x[i,j,k] for j in model.clients_set) - sum(model.x[j,i,k] for j in model.clients_set) == 0
        model.restriccion_3 = Constraint(Model.clients_set, Model.cars_set, rule = restriccion_3)

        #Constraint4: Miller-Tucker-Zemlin restriction to avoid sub-tours
        def restriccion_4(model, i, j, k):
            if i != j:
                return model.u[i,k] - model.u[j,k] + len(model.clients_set)*model.x[i,j,k] <= len(model.clients_set)-1
            else:
                return Constraint.Skip
        model.restriccion_4 = Constraint(model.U, model.clients_set, model.cars_set, rule = restriccion_4)

        #Constraint6: The load of each car should enough to suffice the demand of the clients
        #The load is : sum(self.clients["Demand"][i]*model.x[i,j,k] for i in model.clients_set)/2
        def restriccion_5(model, j, k):
            return sum(self.clients["Product"][i]*model.x[i,j,k] for i in model.clients_set)/2 <= self.cars["Capacity"][k]
        model.restriccion_5 = Constraint(model.clients_set, model.cars_set, rule = restriccion_5)

        #Constraint7: Each car should have enough range to visit all the clients
        def restriccion_6(model, k):
            if self.cars["VehicleType"][k] == "Solar EV" or self.cars["VehicleType"][k] == "Gas Car":
                return self.cars["Range"][k] >= sum(self.car_matrix_distance[i][j]*model.x[i,j,k] for i in model.nodes for j in model.nodes)
            elif self.cars["VehicleType"][k] == "Drone":
                return self.cars["Range"][k] >= sum(self.drone_matrix[i][j]*model.x[i,j,k] for i in model.nodes for j in model.nodes)
        model.restriccion_6 = Constraint(model.cars_set, rule = restriccion_6)


        return model

    def solve_model(self):
        """
        Solve the model using the given solver.
        """
        solver = SolverFactory('highs')
        results = solver.solve(self.model)
        return results

    def display_results(self):
        """
        Display the results of the optimization.
        """
        self.model.display()
    def print_output(self):
        """
        Plot sensor placement solutions on directed graphs using NetworkX for each sensor type in a subplot.
        """
        pass

base_case = BaseCase(gas_cars, drones, solar_ev, car_matrix_distance, car_matrix_time, drone_matrix, clients, num_depots, num_clients, num_cars, cars)

base_case.build_model()

3600
ERROR: Constructing component 'x' from data=None failed:
        KeyboardInterrupt:


KeyboardInterrupt: 